In [5]:
import pandas as pd
import json

In [42]:
total = pd.read_csv("../data/inns_tenderous_check.csv")

In [43]:
total["ogrn"]

0       1127847550566
1       1131650003561
2       1131650008940
3       1131832001806
4       1132901006424
            ...      
3418    1197746695343
3419    1177746281459
3420    1187746027237
3421    1207700380128
3422    1197746604000
Name: ogrn, Length: 3423, dtype: int64

In [44]:
total.columns

Index(['ogrn', 'inn', 'actives_x', 'revenue_x', 'short_term_debt_x',
       'long_term_debt_x', 'name', 'okveds', 'ustavnoi_capital', 'location',
       'egrul_registration_date', 'respondent', 'plaintiffs', 'third',
       'participated', 'is_tenderous', 'region_id', 'email', 'phone'],
      dtype='object')

In [41]:
total["participated"]

0       0
1       0
2       0
3       0
4       0
       ..
3418    1
3419    1
3420    1
3421    1
3422    1
Name: participated, Length: 3423, dtype: int64

In [6]:
with open("../data/phones/phones_all_companies.json") as f:
    phones = json.load(f)

In [8]:
phones_ogrns = [int(ogrn) for ogrn in phones.keys()]

In [10]:
len(set(phones_ogrns).intersection(total["ogrn"]))

3423

In [72]:
age_and_score = [
    (0.5, -20),
    (1, -10),
    (2, 0),
    (3, 5),
    (5, 15),
    (float("inf"), 20)
]

respondent_mean_year_and_score = [
    (0, 10),
    (2, 5),
    (4, 0),
    (10, -5),
    (float("inf"), -10)
]

actives_and_score = [
    (50e3, -10),
    (150e3, -5),
    (300e3, 0),
    (1e6, 5),
    (float("inf"), 10)
]

revenue_after_taxes_relative_and_score = [
    (9, -10),
    (0.5, -5),
    (0.7, 0),
    (0.85, 5),
    (float("inf"), 10)
]


tenders_count_and_score = [
    (0, 0),
    (10, 5),
    (float("inf"), 10)
]


tenders_sum_and_score = [
    (0, 0),
    (float(1e6), 5),
    (float("inf"), 10)
]

def map_for_relation(relation, value):
    for max_val, score_change in relation:
        if value <= max_val:
            return score_change
        
import warnings
from datetime import datetime
import numpy as np
warnings.simplefilter("ignore")
total["tmp_egrul_registration_date"] = pd.to_datetime(total["egrul_registration_date"], errors="coerce")
total["tmp_age"] = (datetime.now() - total["tmp_egrul_registration_date"]) / np.timedelta64(1, 'Y')
total["tmp_respondent_mean_year"] = total["respondent"] / total["age"]
total["tmp_relative_revenue"] = (total["revenue_x"] - total["short_term_debt_x"]) / total["revenue_x"]

cols_and_relations = [
    ("age", age_and_score),
    ("tmp_respondent_mean_year", respondent_mean_year_and_score),
    ("actives_x", actives_and_score),
    ("tmp_relative_revenue", revenue_after_taxes_relative_and_score),
    ("participated", tenders_count_and_score),
    ("tenders_sum_amount", tenders_sum_and_score)
]

score = np.zeros(len(total))

for col, relation in cols_and_relations:
    print("col", col)
    score += total[col].map(lambda age: map_for_relation(relation, age)).values

col age
col tmp_respondent_mean_year
col actives_x
col tmp_relative_revenue
col participated
col tenders_sum_amount


KeyError: 'tenders_sum_amount'

In [77]:
(total["participated"] == 0).sum() / len(total)

0.7630733274905054